In [1]:
import clingo
from gamearg import *
from IPython.display import display, Image
import pandas as pd
import concurrent
from concurrent.futures import ThreadPoolExecutor
import time

## Generating Graph 

With given `node_num`, program will be able to generate all possible edges

In [2]:
def graph_generating(node_num):
    def on_model(m):
        if str(m) == "":
            possible_graphs.append(str(m))
        else:
            elements = str(m).split(" ")
            elements_with_dot = [element + "." for element in elements]
            concatenated = " ".join(elements_with_dot)
            possible_graphs.append(concatenated)

    graph_log = """
    #const n = {}.
    node(1..n).
    {{move(X,Y): node(X), node(Y)}}.
    #show move/2.
    """.format(node_num)
    # print(graph_log)
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    possible_graphs = []
    ctl.solve(on_model=on_model)
    return possible_graphs

In [3]:
possible_graphs = graph_generating(node_num=2)

In [4]:
possible_graphs

['',
 'move(1,2).',
 'move(2,1).',
 'move(1,2). move(2,1).',
 'move(2,2).',
 'move(2,1). move(2,2).',
 'move(1,2). move(2,2).',
 'move(1,2). move(2,1). move(2,2).',
 'move(1,1).',
 'move(1,1). move(1,2).',
 'move(1,1). move(2,2).',
 'move(1,1). move(1,2). move(2,2).',
 'move(1,1). move(2,1).',
 'move(1,1). move(2,1). move(2,2).',
 'move(1,1). move(1,2). move(2,1).',
 'move(1,1). move(1,2). move(2,1). move(2,2).']

## Digest into Game Solver

Based on the `possible_graphs`, we need to digest them into game_solver one by one and ensure that the result will be 3-valued (dir=back and forward)

In [5]:
def nodes_connected(graph):
    graph_log = f"""
    {graph}
    move(X,Y):- move(Y,X).
    node(X):- move(X,_).
    node(X):- move(_,X).
    connected(X,Y) :- move(X,Y).
    connected(X,Y) :- connected(X,Z), move(Z,Y).
    :- node(X), node(Y), not connected(X,Y).
    """
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    return ctl.solve()

In [6]:
columns = [
    "node_number",
    "edge_number",
    "graph",
    "fw_node_wfs",
    "bw_node_wfs",
    "connected",
]
graph_cal = pd.DataFrame(columns=columns)
node_num = 1


def process_graph(graph, node_num):
    try:
        fw_node_wfs_df = node_wfs_cal(graph, "game", reverse=False)
        bw_node_wfs_df = node_wfs_cal(graph, "game", reverse=True)
        return {
            "node_number": node_num,
            "edge_number": len(graph.split(".")) - 1,
            "graph": graph,
            "fw_node_wfs": set(fw_node_wfs_df["wfs"]),
            "bw_node_wfs": set(bw_node_wfs_df["wfs"]),
            "connected": str(nodes_connected(graph)),
        }
    except Exception as e:
        print(f"An error occurred while processing {graph}: {e}")
        return None


data_to_add = []
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for node_num in range(1, 5):
        possible_graphs = graph_generating(node_num)
        if not possible_graphs or all(graph == "" for graph in possible_graphs):
            continue
        for graph in possible_graphs:
            if graph == "":
                continue
            else:
                futures.append(executor.submit(process_graph, graph, node_num))
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            data_to_add.append(result)
graph_cal = pd.DataFrame(data_to_add, columns=columns)

In [7]:
graph_cal

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected
0,3,6,"move(1,2). move(1,3). move(2,1). move(2,2). mo...",{draw},{draw},SAT
1,3,6,"move(1,1). move(1,2). move(1,3). move(2,2). mo...",{draw},{draw},SAT
2,3,5,"move(1,1). move(2,1). move(3,1). move(3,2). mo...",{draw},{draw},SAT
3,3,5,"move(1,1). move(1,2). move(2,3). move(3,2). mo...",{draw},{draw},SAT
4,3,5,"move(1,3). move(2,1). move(2,2). move(2,3). mo...",{draw},{draw},SAT
...,...,...,...,...,...,...
66057,4,14,"move(1,1). move(1,3). move(1,4). move(2,1). mo...",{draw},{draw},SAT
66058,4,15,"move(1,1). move(1,2). move(1,3). move(1,4). mo...",{draw},{draw},SAT
66059,4,15,"move(1,1). move(1,3). move(1,4). move(2,1). mo...",{draw},{draw},SAT
66060,4,15,"move(1,1). move(1,2). move(1,3). move(1,4). mo...",{draw},{draw},SAT


In [11]:
graph_cal["fw_len"] = graph_cal["fw_node_wfs"].apply(len)
graph_cal["bw_len"] = graph_cal["bw_node_wfs"].apply(len)

In [22]:
graph_cal[(graph_cal["fw_len"] == 3) & (graph_cal["bw_len"] == 3)].sort_values(
    ["node_number", "edge_number"]
)

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected,fw_len,bw_len
42,3,2,"move(1,1). move(2,3).","{draw, won, lost}","{draw, won, lost}",UNSAT,3,3
155,3,2,"move(1,3). move(2,2).","{draw, won, lost}","{draw, won, lost}",UNSAT,3,3
161,3,2,"move(1,2). move(3,3).","{draw, won, lost}","{draw, won, lost}",UNSAT,3,3
217,3,2,"move(1,1). move(3,2).","{draw, won, lost}","{draw, won, lost}",UNSAT,3,3
459,3,2,"move(2,1). move(3,3).","{draw, won, lost}","{draw, won, lost}",UNSAT,3,3
...,...,...,...,...,...,...,...,...
58223,4,7,"move(1,1). move(1,2). move(1,4). move(3,1). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
58530,4,7,"move(1,1). move(1,4). move(3,1). move(3,2). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
60975,4,7,"move(1,1). move(1,2). move(1,4). move(3,2). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
61078,4,7,"move(1,1). move(1,4). move(3,2). move(3,4). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3


In [23]:
graph_cal[
    (graph_cal["fw_len"] == 3)
    & (graph_cal["bw_len"] == 3)
    & (graph_cal["connected"] == "SAT")
].sort_values(["node_number", "edge_number"])

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected,fw_len,bw_len
691,4,4,"move(1,2). move(1,4). move(2,2). move(3,1).","{draw, won, lost}","{draw, won, lost}",SAT,3,3
730,4,4,"move(1,4). move(2,1). move(2,2). move(3,1).","{draw, won, lost}","{draw, won, lost}",SAT,3,3
880,4,4,"move(1,3). move(1,4). move(2,2). move(3,2).","{draw, won, lost}","{draw, won, lost}",SAT,3,3
1176,4,4,"move(1,4). move(2,1). move(3,1). move(3,3).","{draw, won, lost}","{draw, won, lost}",SAT,3,3
1199,4,4,"move(1,2). move(1,4). move(2,3). move(3,3).","{draw, won, lost}","{draw, won, lost}",SAT,3,3
...,...,...,...,...,...,...,...,...
58223,4,7,"move(1,1). move(1,2). move(1,4). move(3,1). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
58530,4,7,"move(1,1). move(1,4). move(3,1). move(3,2). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
60975,4,7,"move(1,1). move(1,2). move(1,4). move(3,2). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3
61078,4,7,"move(1,1). move(1,4). move(3,2). move(3,4). mo...","{draw, won, lost}","{draw, won, lost}",SAT,3,3


In [24]:
graph_cal.to_csv("graph_cal_selfloop.csv")